In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DSF/Proj/'DSF Project Fall 2021'/data/
# %cd /content/drive/MyDrive/DSF Project Fall 2021/data

/content/drive/.shortcut-targets-by-id/1uITAlwG_KLn52Rc6b51jPmC6XmnWK3r-/DSF Project Fall 2021/data


In [ ]:
pip install pulp

     |████████████████████████████████| 41.2 MB 1.3 MB/s 


In [ ]:
pip install colorama

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import permutation_test_score as perm_test
from sklearn.metrics import mean_squared_error
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import ttest_ind
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median', add_indicator=True)
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDClassifier
import requests
import pulp
from IPython.display import display, Markdown, Latex
from colorama import Fore, Style

In [ ]:
main_data = pd.read_csv("cleaned_merged_seasons.csv")
data = main_data.copy()
players =data['name'].unique()

print(type(data))

<class 'pandas.core.frame.DataFrame'>


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
merged_gw_21_22 = pd.read_csv("merged_season_2021-22.csv")

In [ ]:
data

,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15T19:00:00Z,0,4,Chelsea,0,0,0,0,1,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,1,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13T14:00:00Z,0,13,Southampton,0,0,0,0,1,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
2,2,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13T14:00:00Z,69,14,Stoke,0,0,0,0,1,0,2723,1.0,1.0,0.0,1,0,0,0,45,True,1,1
3,3,2016-17,Adam Lallana,MID,NaN,1,2,33,0,33.7,205,8,3,1,14.2,51.2,2016-08-14T15:00:00Z,75,1,Arsenal,0,0,0,0,1,0,155525,4.0,3.0,57.0,11,0,0,0,70,False,1,1
4,4,2016-17,Adrián San Miguel del Castillo,GK,NaN,0,0,16,0,0.0,450,10,2,0,3.0,29.8,2016-08-15T19:00:00Z,90,4,Chelsea,0,0,0,0,1,4,94480,1.0,2.0,0.0,2,0,0,0,50,False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69019,69019,2020-21,Aaron Ramsdale,GK,Sheffield Utd,0,2,30,1,10.0,483,376,0,0,3.2,22.0,2021-05-23T15:00:00Z,90,4,Burnley,0,0,0,0,38,3,162985,0.0,1.0,0.0,9,13618,17381,3763,46,True,0,38
69020,69020,2020-21,Adrien Silva,MID,Leicester,0,0,0,0,0.0,220,373,0,0,0.0,0.0,2021-05-23T15:00:00Z,0,17,Spurs,0,0,0,0,38,0,16396,4.0,2.0,0.0,0,-16,0,16,44,True,0,38
69021,69021,2020-21,Sidnei Tavares,MID,Leicester,0,0,0,0,0.0,676,373,0,0,0.0,0.0,2021-05-23T15:00:00Z,0,17,Spurs,0,0,0,0,38,0,337,4.0,2.0,0.0,0,32,60,28,44,True,0,38
69022,69022,2020-21,Kalvin Phillips,MID,Leeds,0,0,17,0,12.8,204,372,1,1,6.1,40.6,2021-05-23T15:00:00Z,90,18,West Brom,0,0,0,0,38,0,74008,1.0,3.0,8.0,6,210,789,579,49,True,1,38


In [ ]:
def damp(grouped_df,col,damping=0.7):
  new_df = pd.DataFrame(data = {'name':grouped_df['name'].unique()}) 
  for player in new_df['name'].unique():
    val = 0;
    for season in ['2016-17','2017-18','2018-19','2019-20','2020-21']:
      df = grouped_df[((grouped_df.name == player) & (grouped_df.season_x == season))]
      
      if season == '2016-17':
        if len(df[col].values) == 0:
          val =   0
        else:
          val = df[col].values[0]
        continue;
      
      if len(df[col].values) == 0:
        val = (val * damping + 0) / (1+damping)
      else:
        val = (val * damping + df[col].values[0]) / (1+damping)
        # if col == 'goals_conceded' and player == 'Hamza Choudhury':
        #   print(val)
    new_df.loc[new_df.name == player,[col]] = val


  return new_df

In [ ]:
data['minutes'].value_counts()

0     32231
90    23026
45      876
1       516
77      251
      ...  
43       13
49       12
40       11
44       10
46        8
Name: minutes, Length: 91, dtype: int64

In [ ]:
# data = data[data['name'] == 'Hamza Choudhury']
data = data[(data['minutes']>0)]
final_data = pd.DataFrame(data = {'name':data['name'].unique()}) 


# for i in ['assists','goals_scored','goals_conceded','ict_index','influence','creativity', 'threat','value','saves']:
#adding minutes and penalties_missed to the damping
for i in ['assists','goals_scored','goals_conceded','ict_index','influence','creativity', 'threat','value','saves','minutes','penalties_saved']:
  temp_df_mean = data.groupby(['name','season_x']).mean().reset_index()
  val = damp(temp_df_mean,i)
  for player in data['name'].unique():
    final_data.loc[final_data.name == player,[i]] = val[val.name == player][i].values[0]

# temp_df_sum = data.groupby(['name','season_x']).sum().reset_index()
# temp_df_sum['minutes'] = temp_df_sum['minutes']/38 
# final_data['minutes'] =  damp(temp_df_sum,'minutes')['minutes']
 
# final_data
# final_data[final_data['name'] == 'Hamza Choudhury']


KeyboardInterrupt: ignored

In [ ]:
x= data[(data['minutes']>0)]
y =x.groupby(['name','season_x']).sum().reset_index()
y.sort_values(['goals_scored'],ascending=False)

In [ ]:
x= data[(data['minutes']>0)]
y =x.groupby(['name','season_x']).count().reset_index()
y.sort_values(['goals_scored'],ascending=False)
x_kane = x[x['name']=="Mohamed Salah"]
x_kane = x_kane.groupby(['season_x']).mean().reset_index()
x_kane

In [ ]:
final_data
x= data[(data['minutes']>0)]
y =x.groupby(['name','season_x']).mean().reset_index()
y.sort_values(['goals_scored'],ascending=False)

In [ ]:
final_data[final_data['name'] == 'Hamza Choudhury']

In [ ]:
# bonus

for player in data['name'].unique():
  value = data[data['name'] == player]['bonus'].value_counts()
  player_mapping = {}
  for i in [0,1,2,3]:
    player_mapping[i] = 0
  for i in value.index.tolist():
    player_mapping[i] = value[i] / sum(value.to_list())

  p_arr = []
  for i in [0,1,2,3]:
    p_arr.append(player_mapping[i])
  # print(sum(p_arr))
  final_data.loc[final_data.name == player,['bonus']] = np.random.choice(np.arange(0, 4), p=p_arr)


# Kenny Tete
# x = data[data['name'] == "Kenny Tete"]['bonus'].value_counts()
# x[2]

In [ ]:
# yellow_cards
for player in data['name'].unique():
  value = data[data['name'] == player]['yellow_cards'].value_counts()
  player_mapping = {}
  for i in [0,1]:
    player_mapping[i] = 0
  for i in value.index.tolist():
    player_mapping[i] = value[i] / sum(value.to_list())

  p_arr = []
  for i in [0,1]:
    p_arr.append(player_mapping[i])

  final_data.loc[final_data.name == player,['yellow_cards']] = np.random.choice(np.arange(0, 2), p=p_arr)
final_data['yellow_cards'].value_counts()

In [ ]:
# red_cards
for player in data['name'].unique():
  value = data[data['name'] == player]['red_cards'].value_counts()
  player_mapping = {}
  for i in [0,1]:
    player_mapping[i] = 0
  for i in value.index.tolist():
    player_mapping[i] = value[i] / sum(value.to_list())

  p_arr = []
  for i in [0,1]:
    p_arr.append(player_mapping[i])

  final_data.loc[final_data.name == player,['red_cards']] = np.random.choice(np.arange(0, 2), p=p_arr)
final_data['red_cards'].value_counts()

In [ ]:
# clean sheets
for player in data['name'].unique():
  value = data[data['name'] == player]['clean_sheets'].value_counts()
  player_mapping = {}
  for i in [0,1]:
    player_mapping[i] = 0
  for i in value.index.tolist():
    player_mapping[i] = value[i] / sum(value.to_list())

  p_arr = []
  for i in [0,1]:
    p_arr.append(player_mapping[i])

  final_data.loc[final_data.name == player,['clean_sheets']] = np.random.choice(np.arange(0, 2), p=p_arr)
final_data['clean_sheets'].value_counts()

In [ ]:
# final_data['bfs'] = data.groupby("name").median().reset_index()['bfs']
x = data.groupby("name").median().reset_index()
final_data['bps'] = x['bps']
final_data

In [ ]:
final_data.sort_values(['minutes'],ascending=False).head(20)

In [ ]:
#adding position to the final_data df
# final_data2 = final_data
for player in final_data['name'].unique():
  final_data.loc[final_data.name == player,['position']] = data[data.name == player]['position'].values[0]

final_data

In [ ]:
# data2 = data[data['season_x'] == '2021-22']
# data2 = data2[data2['GW'] == 1]
# for player in final_data['name'].unique():
#   final_data.loc[final_data.name == player,['total_points']] = data2[data2.name == player]['total_points']

In [ ]:
merged_gw_21_22.columns

In [ ]:
data2 = merged_gw_21_22.copy()
data2 = data2[data2['GW'] == 1]
for player in final_data['name'].unique():
  if len(data2[data2.name == player]['total_points'].values) != 0:
    final_data.loc[final_data.name == player,['total_points']] = data2[data2.name == player]['total_points'].values[0]

In [ ]:
final_data['total_points'] = final_data['total_points'].fillna(0)

In [ ]:
final_data

In [ ]:
np.random.choice(np.arange(1, 7), p=[0.1, 0.05, 0.05, 0.2, 0.4, 0.2])

# Score Prediction

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import permutation_test_score as perm_test
from sklearn.metrics import mean_squared_error
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import ttest_ind
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median', add_indicator=True)
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDClassifier
import requests
from sklearn.metrics import mean_squared_error
import math
import warnings
from sklearn.preprocessing import MinMaxScaler

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
merged_data_df = pd.read_csv('cleaned_merged_seasons.csv')

In [ ]:
merged_data_df = merged_data_df[merged_data_df.minutes >0]

In [ ]:
gk_player_df = merged_data_df[merged_data_df.position == 'GK']
mid_player_df = merged_data_df[merged_data_df.position == 'MID']
def_player_df = merged_data_df[merged_data_df.position == 'DEF']
fwd_player_df = merged_data_df[merged_data_df.position == 'FWD']

In [ ]:
cols = ['total_points','season_x', 'name', 'minutes','xP','bonus','assists','bps','clean_sheets','goals_conceded','goals_scored','ict_index','influence','threat','value','yellow_cards','creativity']

# gk_player_df_new = gk_player_df[['total_points','season_x', 'name', 'minutes','bonus','bps','clean_sheets','goals_conceded','goals_scored','ict_index','influence','threat','value','yellow_cards','creativity']]
# mid_player_df_new = mid_player_df[['total_points','season_x', 'name', 'minutes','bonus','assists','bps','clean_sheets','goals_scored','ict_index','influence','threat','value','yellow_cards','creativity']]
# def_player_df_new = def_player_df[['total_points','season_x', 'name', 'minutes','bonus','assists','bps','clean_sheets','goals_conceded','goals_scored','ict_index','influence','threat','value','yellow_cards','creativity']]
# fwd_player_df_new = fwd_player_df[['total_points','season_x', 'name', 'minutes','bonus','assists','bps','goals_scored','ict_index','influence','threat','value','yellow_cards','creativity']]

# gk_player_df_new = gk_player_df[['total_points','season_x', 'name', 'minutes','bonus','bps','clean_sheets','goals_conceded','goals_scored','ict_index','value','yellow_cards','red_cards']]

gk_player_df_new = gk_player_df[['total_points','season_x', 'name', 'minutes','bonus','bps','clean_sheets','goals_conceded','value','saves','yellow_cards']]
mid_player_df_new = mid_player_df[['total_points','season_x', 'name', 'minutes','bonus','assists','bps','goals_scored','ict_index','value','yellow_cards','red_cards', 'threat', 'influence']]
def_player_df_new = def_player_df[['total_points','season_x', 'name', 'minutes','bonus','assists','bps','clean_sheets','goals_conceded','goals_scored','threat','value','yellow_cards','red_cards','influence']]
fwd_player_df_new = fwd_player_df[['total_points','season_x', 'name', 'minutes','bonus','assists','bps','goals_scored','ict_index','value','yellow_cards','red_cards', 'threat', 'influence']]

In [ ]:
# gk_player_df_train = gk_player_df_new[~(gk_player_df_new['season_x'] == '2021-22')]
gk_player_df_train = gk_player_df_new[~(gk_player_df_new.season_x.isin(['2015-16','2016-17','2021-22']))]
gk_player_df_test = gk_player_df_new[gk_player_df_new['season_x'] == '2021-22']
mid_player_df_train = mid_player_df_new[~(mid_player_df_new.season_x.isin(['2015-16','2016-17','2021-22']))]
mid_player_df_test = mid_player_df_new[mid_player_df_new['season_x'] == '2021-22']
def_player_df_train = def_player_df_new[~(def_player_df_new.season_x.isin(['2015-16','2016-17','2021-22']))]
def_player_df_test = def_player_df_new[def_player_df_new['season_x'] == '2021-22']
fwd_player_df_train = fwd_player_df_new[~(fwd_player_df_new.season_x.isin(['2015-16','2016-17','2021-22']))]
fwd_player_df_test = fwd_player_df_new[fwd_player_df_new['season_x'] == '2021-22']

In [ ]:
gk_player_df_train_att = gk_player_df_train.drop(['season_x', 'name','total_points'], axis=1)
mid_player_df_train_att = mid_player_df_train.drop(['season_x', 'name','total_points'], axis=1)
def_player_df_train_att = def_player_df_train.drop(['season_x', 'name','total_points'], axis=1)
fwd_player_df_train_att = fwd_player_df_train.drop(['season_x', 'name','total_points'], axis=1)

gk_player_df_train_score = gk_player_df_train[['total_points']]
mid_player_df_train_score = mid_player_df_train[['total_points']]
def_player_df_train_score = def_player_df_train[['total_points']]
fwd_player_df_train_score = fwd_player_df_train[['total_points']]

In [ ]:
gk_reg_model_rf = RandomForestRegressor(n_estimators=30, max_depth=15, criterion='mse')
gk_reg_model_rf.fit(gk_player_df_train_att, gk_player_df_train_score.total_points)
def_reg_model_rf = RandomForestRegressor(n_estimators=30, max_depth=15, criterion='mse')
def_reg_model_rf.fit(def_player_df_train_att, def_player_df_train_score.total_points)
mid_reg_model_rf = RandomForestRegressor(n_estimators=30, max_depth=15, criterion='mse')
mid_reg_model_rf.fit(mid_player_df_train_att, mid_player_df_train_score.total_points)
fwd_reg_model_rf = RandomForestRegressor(n_estimators=30, max_depth=15, criterion='mse')
fwd_reg_model_rf.fit(fwd_player_df_train_att, fwd_player_df_train_score.total_points)

In [ ]:
# plt.figure(figsize=(30,20))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# mask = np.zeros_like(gk_player_df.corr(), dtype='bool')
# mask[np.triu_indices_from(mask)] = True
# sns.heatmap(data=gk_player_df.corr(),annot=True, cmap=cmap, fmt='.3f')

In [ ]:
# plt.figure(figsize=(30,20))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# mask = np.zeros_like(mid_player_df.corr(), dtype='bool')
# mask[np.triu_indices_from(mask)] = True
# sns.heatmap(data=mid_player_df.corr(),annot=True, cmap=cmap, fmt='.3f')

In [ ]:
# plt.figure(figsize=(30,20))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# mask = np.zeros_like(def_player_df.corr(), dtype='bool')
# mask[np.triu_indices_from(mask)] = True
# sns.heatmap(data=def_player_df.corr(),annot=True, cmap=cmap, fmt='.3f')

In [ ]:
# plt.figure(figsize=(30,20))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# mask = np.zeros_like(fwd_player_df.corr(), dtype='bool')
# mask[np.triu_indices_from(mask)] = True
# sns.heatmap(data=fwd_player_df.corr(),annot=True, cmap=cmap, fmt='.3f')

# Using Ashley's weighted damped data from here

In [ ]:
final_data_copy = final_data

In [ ]:
final_data = final_data_copy

In [ ]:
final_data_gk = final_data[final_data.position == 'GK']
final_data_mid = final_data[final_data.position == 'MID']
final_data_def = final_data[final_data.position == 'DEF']
final_data_fwd = final_data[final_data.position == 'FWD']

In [ ]:
# final_data_mid['goals_scored'] = final_data_mid['goals_scored'] * 3.5
# final_data_mid['assists'] = final_data_mid['assists'] * 3.5
# final_data_fwd['goals_scored'] = final_data_fwd['goals_scored'] * 1.5
# final_data_fwd['assists'] = final_data_fwd['assists'] * 1.5
# final_data_def['assists'] = final_data_def['assists'] * 2.5
# final_data_gk['goals_conceded'] = final_data_gk['goals_conceded'] * 2.5
# final_data_gk['saves'] = final_data_gk['saves'] * 0.6

In [ ]:
final_data_gk

In [ ]:
final_data_gk['predicted_score_rf'] = gk_reg_model_rf.predict(final_data_gk[['minutes','bonus','bps','clean_sheets','goals_conceded','saves','value','yellow_cards']])
final_data_mid['predicted_score_rf'] = mid_reg_model_rf.predict(final_data_mid[['minutes','bonus','assists','bps','goals_scored','ict_index','value','yellow_cards','red_cards', 'threat', 'influence']])
final_data_def['predicted_score_rf'] = def_reg_model_rf.predict(final_data_def[['minutes','bonus','assists','bps','clean_sheets','goals_conceded','goals_scored','threat','value','yellow_cards','red_cards', 'influence']])
final_data_fwd['predicted_score_rf'] = fwd_reg_model_rf.predict(final_data_fwd[['minutes','bonus','assists','bps','goals_scored','ict_index','value','yellow_cards','red_cards', 'threat', 'influence']])

In [ ]:
#normalizing scores based on position
final_data_gk['predicted_score_rf'] = final_data_gk['predicted_score_rf'] * 0.65
# final_data_mid['predicted_score_rf'] = final_data_mid['predicted_score_rf'] * 1.85
# final_data_def['predicted_score_rf'] = final_data_def['predicted_score_rf'] * 1.2
# final_data_fwd['predicted_score_rf'] = final_data_fwd['predicted_score_rf'] * 1.15

In [ ]:
temp123 = final_data_fwd[final_data_fwd['name'].str.contains('Kane')]
temp123

In [ ]:
final_data[final_data.name.str.contains('Gea')]

In [ ]:
tenptest = mid_player_df_test[mid_player_df_test['name'].str.contains('Salah')]
tenptest

In [ ]:
temptest2 =  merged_data_df[merged_data_df['season_x'] == '2021-22']
temptest2 = temptest2[temptest2['name'].str.contains('Nick')]
temptest2

In [ ]:
final_predicted_scores_df = pd.concat([final_data_gk, final_data_mid, final_data_def, final_data_fwd])

In [ ]:
top50_scoring_players = final_predicted_scores_df.sort_values(['predicted_score_rf'],ascending=False).head(50)[['name','position', 'predicted_score_rf']]
top50_scoring_players

In [ ]:
top50_scoring_players_count = top50_scoring_players.groupby(['position']).count().reset_index()
top50_scoring_players_count['percent'] = top50_scoring_players_count['name'] * 2
top50_scoring_players_count = top50_scoring_players_count.drop(['name','predicted_score_rf'], axis=1)
top50_scoring_players_count

In [ ]:
testactualranks = merged_data_df.sort_values(['total_points'],ascending=False)[['name','position', 'total_points','GW']]
testactualranks = testactualranks[testactualranks['GW'] == 1]
testactualranks.sort_values(['total_points'],ascending=False).head(30)

In [ ]:
final_predicted_scores_df_temp2 = final_predicted_scores_df
final_predicted_scores_df_temp2 = final_predicted_scores_df_temp2[['name', 'position', 'predicted_score_rf']]

merged_data_df_temp2 = merged_data_df[merged_data_df['GW'] == 1]
for player in merged_data_df_temp2['name'].unique():
  final_predicted_scores_df_temp2.loc[final_predicted_scores_df_temp2.name == player,['total_points']] = merged_data_df_temp2[merged_data_df_temp2.name == player]['total_points'].values[0]

final_predicted_scores_df_temp2.sort_values(['total_points'],ascending=False).head(30)

In [ ]:
def convertPosToNum(pos):
  #print(type(pos))
  # print(pos)
  if pos == 'GK':
    return 1
  elif pos == 'MID':
    return 3
  elif pos == 'DEF':
    return 2
  else:
    return 4

In [ ]:
#https://medium.com/@joseph.m.oconnor.88/linearly-optimising-fantasy-premier-league-teams-part-2-2bd9c3b3d34e
def select_team(predicted_Scores, player_price, player_positions, clubs, total_budget=100, sub_factor=0.6):
    num_players = len(predicted_Scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    lp_decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    lp_sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    lp_captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    model += sum((lp_captain_decisions[i] + lp_decisions[i] + lp_sub_decisions[i]*sub_factor) * predicted_Scores[i]
                 for i in range(num_players)), "Objective"

    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 2) >= 3
    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 2) <= 5
    model += sum(lp_decisions[i] + lp_sub_decisions[i] for i in range(num_players) if player_positions[i] == 2) == 5

    model += sum((lp_decisions[i] + lp_sub_decisions[i]) * player_price[i] for i in range(num_players)) <= total_budget 

    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 1) == 1
    model += sum(lp_decisions[i] + lp_sub_decisions[i] for i in range(num_players) if player_positions[i] == 1) == 2

    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 2) >= 3
    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 2) <= 5
    model += sum(lp_decisions[i] + lp_sub_decisions[i] for i in range(num_players) if player_positions[i] == 2) == 5

    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 4) >= 1
    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 4) <= 3

    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 3) >= 3
    model += sum(lp_decisions[i] for i in range(num_players) if player_positions[i] == 3) <= 5
    model += sum(lp_decisions[i] + lp_sub_decisions[i] for i in range(num_players) if player_positions[i] == 3) == 5




    model += sum(lp_decisions[i] + lp_sub_decisions[i] for i in range(num_players) if player_positions[i] == 4) == 3

    for club_id in np.unique(clubs):
        model += sum(lp_decisions[i] + lp_sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3 

    model += sum(lp_decisions) == 11  
    model += sum(lp_captain_decisions) == 1  
    
    for i in range(num_players):  
        model += (lp_decisions[i] - lp_captain_decisions[i]) >= 0  
        model += (lp_decisions[i] + lp_sub_decisions[i]) <= 1  

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return lp_decisions, lp_captain_decisions, lp_sub_decisions

In [ ]:
# eval_main_data =pd.read_csv("2020-21/gws/gw1.csv")

eval_main_data = merged_gw_21_22
eval_data = eval_main_data

In [ ]:
final_predicted_scores_df
for player in final_predicted_scores_df['name'].unique():
  for i in ['team','position']:

    if len(eval_data[eval_data.name == player][i].values) != 0:
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,[i]] = eval_data[eval_data.name == player][i].values[0]


for player in final_predicted_scores_df['name'].unique():

  if len(eval_data[eval_data.name == player]['total_points'].values) != 0:
    final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['real_points']] = eval_data[eval_data.name == player]['total_points'].values[0]

In [ ]:
# df_mean = final_predicted_scores_df.dropna(subset=['team'])
# df_mean['team'] = df_mean['team'].astype(dtype="category")
# df_mean['team'] =  df_mean.team.cat.codes
# df_mean['position'] = df_mean['position'].apply(convertPosToNum)
# df_mean

In [ ]:
# expected_scores = df_mean["predicted_score_rf"]  # total points from last season
# prices = df_mean["value"]/10
# positions = df_mean["position"]
# clubs = df_mean["team"]
# real_scores = df_mean['real_points']
# # so we can read the results
# names = df_mean["name"]
# print(df_mean.dtypes)
# decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)

In [ ]:
def convertNumToPos(pos):
  #print(type(pos))
  # print(pos)
  if pos == 1:
    return 'GK'
  elif pos == 3:
    return 'MID'
  elif pos == 2:
    return 'DEF'
  else:
    return 'FWD'

In [ ]:
# print(f'{Fore.LIGHTBLUE_EX}{Style.BRIGHT}Our Final playing XI as predicted by the model:')


# names_list = []
# curr_budget = 0
# player_indices = []
# total_points = {}
# tpoints= 0
# rpoints =0
# for i in range(df_mean.shape[0]):
#     if decisions[i].value() != 0:
#         names_list.append(names.iloc[i])
#         total_points[names.iloc[i]] = expected_scores.iloc[i]
#         player_indices.append(i)
#         curr_budget+= prices.iloc[i]
#         tpoints += expected_scores.iloc[i]
#         rpoints += real_scores.iloc[i]
#         display(Markdown("**Name**= *{}*, **Expected Points** = {},**Real Points** = {}, **Cost** = {}M$, **Position** = {}".format(names.iloc[i], expected_scores.iloc[i],real_scores.iloc[i], prices.iloc[i],  convertNumToPos(positions.iloc[i]))))

# print("")
# print(f'{Fore.LIGHTBLUE_EX}{Style.BRIGHT}Captain:')
# for i in range(df_mean.shape[0]):
#     if captain_decisions[i].value() == 1:
#         display(Markdown("**Name**= *{}*, **Expected Points** = {},**Real Points** = {}, **Cost** = {}M$, **Position** = {}".format(names.iloc[i], expected_scores.iloc[i],real_scores.iloc[i], prices.iloc[i],  convertNumToPos(positions.iloc[i]))))

# print("")
# print(f'{Fore.LIGHTBLUE_EX}{Style.BRIGHT}List of Substitutes:')

# for i in range(df_mean.shape[0]):
#     if sub_decisions[i].value() == 1:
#         names_list.append(names.iloc[i])
#         player_indices.append(i)
#         curr_budget+= prices.iloc[i]
#         # tpoints += expected_scores.iloc[i]
#         display(Markdown("**Name**= *{}*, **Expected Points** = {},**Real Points** = {}, **Cost** = {}M$, **Position** = {}".format(names.iloc[i], expected_scores.iloc[i],real_scores.iloc[i], prices.iloc[i],  convertNumToPos(positions.iloc[i]))))

In [ ]:
# print(rpoints)
# tpoints

In [ ]:
# merged_data_df[merged_data_df['name'].str.contains('Gea')]
final_predicted_scores_df.columns

In [ ]:
final_predicted_scores_df.sort_values(['goals_scored'],ascending=False).head(30)

In [ ]:
ndata = main_data.copy()
ndata[(ndata['name'] == 'Sergio Agüero') & (ndata['season_x'] == '2021-22')].sort_values(['GW'],ascending=True).head(30)

In [ ]:
final_main_data = main_data.copy()

In [ ]:
main_data[main_data.name.str.contains('Gea')]

In [ ]:
new_players = merged_gw_21_22['name'].unique()
final_main_data = final_main_data[final_main_data['name'].isin(new_players)]
final_predicted_scores_df = final_predicted_scores_df[final_predicted_scores_df['name'].isin(new_players)]

In [ ]:
final_predicted_scores_df.columns

In [ ]:
merged_gw_21_22[merged_gw_21_22.GW == 1]

In [ ]:
final_predicted_scores_df.columns

In [ ]:
temp = merged_gw_21_22[(merged_gw_21_22['name'] == 'David de Gea') &(merged_gw_21_22['GW'] == 1)]
temp['value'].values[0]

In [ ]:
for player in final_predicted_scores_df['name'].unique():
    # print(player)
    temp = merged_gw_21_22[(merged_gw_21_22['name'] == player) &(merged_gw_21_22['GW'] == 1)]
    # print(temp)
    if len(temp)!=0:
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['value']] = temp['value'].values[0]
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['position']] = temp['position'].values[0]
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['team']] = temp['team'].values[0]
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['ict_index']] = temp['ict_index'].values[0]
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['influence']] = temp['influence'].values[0]
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['creativity']] = temp['creativity'].values[0]
      final_predicted_scores_df.loc[final_predicted_scores_df.name == player,['threat']] = temp['threat'].values[0]


In [ ]:
old_players = final_predicted_scores_df['name'].unique()
for player in new_players:
  if player not in old_players:
    temp = merged_gw_21_22[merged_gw_21_22['name'] == player]
    pos = temp['position'].values[0]
    team = temp['team'].values[0]
    value_player = temp['value'].values[0]
    ict_player = temp['ict_index'].values[0] 
    influence_player = temp['influence'].values[0] 
    creativity_player = temp['creativity'].values[0] 
    threat_player = temp['threat'].values[0] 

    dict = {'name':player, 'assists':0, 'goals_scored':0, 'goals_conceded':0, 'ict_index':ict_player,
       'influence':influence_player, 'creativity':creativity_player, 'threat':threat_player, 'value':value_player, 'saves':0, 'minutes':45,
       'penalties_saved':0, 'bonus':0, 'yellow_cards':0, 'red_cards':0, 'clean_sheets':0,
       'bps':0, 'position':pos, 'total_points':0, 'predicted_score_rf':0, 'team':team,
       'real_points':0}

    #append new players rows to df
    final_main_data = final_main_data.append(dict, ignore_index = True)
    final_predicted_scores_df = final_predicted_scores_df.append(dict, ignore_index=True) 

In [ ]:
main_final_predicted_scores_df = final_predicted_scores_df.copy()
main_final_predicted_scores_df['GW'] = 1

In [ ]:
main_final_predicted_scores_df

In [ ]:
final_predicted_scores_df.to_pickle('final_predicted_scores_df_nov_30.pkl')

In [ ]:


final_main_data_prob = main_final_predicted_scores_df.loc[:, ['name']]

# bonus
for player in final_main_data['name'].unique():
  count_bonus = final_main_data[final_main_data['name'] == player]['bonus'].value_counts()
  count_clean_sheets = final_main_data[final_main_data['name'] == player]['clean_sheets'].value_counts()
  count_red = final_main_data[final_main_data['name'] == player]['red_cards'].value_counts()
  count_yellow = final_main_data[final_main_data['name'] == player]['yellow_cards'].value_counts()
  player_mapping_bonus = {}

  for i in [0,1,2,3]:
    player_mapping_bonus[i] = 0
  for i in count_bonus.index.tolist():
    player_mapping_bonus[i] = count_bonus[i] / sum(count_bonus.to_list())
    final_main_data_prob.loc[final_main_data_prob.name == player,['bonus'+str(i)]] = player_mapping_bonus[i]
  
  final_main_data_prob.loc[final_main_data_prob.name == player,['yellow']] = count_yellow[1] / sum(count_yellow.to_list()) if count_yellow.shape[0] == 2 else 0.0
  final_main_data_prob.loc[final_main_data_prob.name == player,['red']] = count_red[1] / sum(count_red.to_list()) if count_red.shape[0] == 2 else 0.0
  final_main_data_prob.loc[final_main_data_prob.name == player,['clean_sheets']] = count_clean_sheets[1] / sum(count_clean_sheets.to_list()) if count_clean_sheets.shape[0] == 2 else 0.0


final_main_data_prob = final_main_data_prob.fillna(0)

In [ ]:
def getProbValue(colName, playerName):

  final_main_data_prob_currentPlayer = final_main_data_prob[final_main_data_prob['name'] == playerName]

  if len(final_main_data_prob_currentPlayer['name']) == 0:
    return 0
  p_arr = []
  if colName == 'bonus':
    for i in [0,1,2,3]:
      p_arr.append(final_main_data_prob_currentPlayer['bonus'+str(i)].values[0])
    return np.random.choice(np.arange(0, 4), p=p_arr)
  elif colName == 'yellow_cards':
    p_arr.append(1 - final_main_data_prob_currentPlayer['yellow'].values[0])
    p_arr.append(final_main_data_prob_currentPlayer['yellow'].values[0])
    return np.random.choice(np.arange(0, 2), p=p_arr)
  elif colName == 'red_cards':
    p_arr.append(1 - final_main_data_prob_currentPlayer['red'].values[0])
    p_arr.append(final_main_data_prob_currentPlayer['red'].values[0])
    return np.random.choice(np.arange(0, 2), p=p_arr)
  elif colName == 'clean_sheets':
    p_arr.append(1 - final_main_data_prob_currentPlayer['clean_sheets'].values[0])
    p_arr.append(final_main_data_prob_currentPlayer['clean_sheets'].values[0])
    return np.random.choice(np.arange(0, 2), p=p_arr)

In [ ]:
merged_gw_21_22.columns

In [ ]:
main_final_predicted_scores_df.columns

In [ ]:
main_final_predicted_scores_df = main_final_predicted_scores_df.drop(['real_points'], axis=1)

In [ ]:
def getWeightedValue(gw):
  df = merged_gw_21_22.copy()
  df2 = df[df['GW'] == gw]
  if gw<6:
    df = df[df['GW']<gw]
    #df2 = df[df['GW'] == gw]
  else:
    df = df[(df['GW']<gw) & (df['GW']>= (gw-5))]
  df = df.groupby(['name']).mean().reset_index()
  new_df =  df.copy()
  for player in df['name'].unique():
    for i in ['assists','goals_scored','goals_conceded','ict_index','influence','creativity', 'threat','saves','minutes','penalties_saved']:
      if len(final_predicted_scores_df[final_predicted_scores_df.name == player][i].values):
        new_df.loc[new_df.name == player,[i]] = df[df.name == player][i].values[0] * 0.35 + 0.65 *final_predicted_scores_df[final_predicted_scores_df.name == player][i].values[0]
    for i in ['red_cards','yellow_cards','bonus','clean_sheets']:
      new_df.loc[new_df.name == player,[i]] = getProbValue(i,player)
    new_df.loc[new_df.name == player,['GW']] = gw
    # print(df2[(df2.name == player)]['total_points'].values[0])
    new_df.loc[new_df.name == player,['total_points']] = df2[(df2.name == player)]['total_points'].values[0]
    #print(df2[(df2.name == player)]['value'].values[0])
    new_df.loc[new_df.name == player,['value']] = df2[(df2.name == player)]['value'].values[0]

  new_df = new_df[['name', 'assists', 'goals_scored', 'goals_conceded', 'ict_index',
       'influence', 'creativity', 'threat', 'value', 'saves', 'minutes',
       'penalties_saved', 'bonus', 'yellow_cards', 'red_cards', 'clean_sheets',
       'bps', 'total_points', 'GW']]

  return new_df

In [ ]:
getWeightedValue(7)

In [ ]:
main_final_predicted_scores_df_copy = main_final_predicted_scores_df.copy()
main_player_pos = main_final_predicted_scores_df_copy[['name', 'position', 'team']]

In [ ]:
for gw in range(2,13):
  generated_df = getWeightedValue(gw)
  generated_df = generated_df.merge(main_player_pos, how='left', on ='name')

  data_gk = generated_df[generated_df.position == 'GK']
  data_mid = generated_df[generated_df.position == 'MID']
  data_def = generated_df[generated_df.position == 'DEF']
  data_fwd = generated_df[generated_df.position == 'FWD']

  data_gk['predicted_score_rf'] = gk_reg_model_rf.predict(data_gk[['minutes','bonus','bps','clean_sheets','goals_conceded','saves','value','yellow_cards']])
  data_mid['predicted_score_rf'] = mid_reg_model_rf.predict(data_mid[['minutes','bonus','assists','bps','goals_scored','ict_index','value','yellow_cards','red_cards', 'threat', 'influence']])
  data_def['predicted_score_rf'] = def_reg_model_rf.predict(data_def[['minutes','bonus','assists','bps','clean_sheets','goals_conceded','goals_scored','threat','value','yellow_cards','red_cards', 'influence']])
  data_fwd['predicted_score_rf'] = fwd_reg_model_rf.predict(data_fwd[['minutes','bonus','assists','bps','goals_scored','ict_index','value','yellow_cards','red_cards', 'threat', 'influence']])

  #normalizing scores based on position
  data_gk['predicted_score_rf'] = data_gk['predicted_score_rf'] * 0.65
  # data_gk['predicted_score_rf'] = data_gk['predicted_score_rf'] * 0.65
  # data_mid['predicted_score_rf'] = data_mid['predicted_score_rf'] * 1.85
  # data_def['predicted_score_rf'] = data_def['predicted_score_rf'] * 1.2
  # data_fwd['predicted_score_rf'] = data_fwd['predicted_score_rf'] * 1.15

  final_generated_df = pd.concat([data_gk, data_mid, data_def, data_fwd])

  main_final_predicted_scores_df = pd.concat([main_final_predicted_scores_df, final_generated_df])
  
    

In [ ]:
main_final_predicted_scores_df

In [ ]:
from numpy import nan

In [ ]:
xy = main_final_predicted_scores_df.copy()
xy[xy['total_points'].isnull()]


In [ ]:
# main_final_predicted_scores_df[main_final_predicted_scores_df.name.str.contains('Gea')]
# merged_gw_21_22
# merged_gw_21_22[merged_gw_21_22.name.str.contains('Gea')]
main_final_predicted_scores_df[main_final_predicted_scores_df['position'] == 'GK']

In [ ]:
main_final_predicted_scores_df.sort_values(['predicted_score_rf'],ascending=False).head(50)[['name','position', 'predicted_score_rf','goals_scored','assists','GW']]

In [ ]:
import pickle
main_final_predicted_scores_df.to_pickle('ashley_nov29.pkl')

In [ ]:
final_ultimate_data_df = pd.read_pickle('ashley_nov29.pkl')
final_ultimate_data_df

In [ ]:
p_name = 'Mohamed Salah'
# final_ultimate_data_df[(final_ultimate_data_df.GW == 1) & (final_ultimate_data_df.name == p_name) ]

merged_gw_21_22[(merged_gw_21_22.GW == 2) & (merged_gw_21_22.name == p_name) ]

In [ ]:
merged_gw_21_22.xP.isna().sum()

In [ ]:
final_ultimate_data_df

In [ ]:
final_decisions =[]
for i in range(1,13):

  working_df = final_ultimate_data_df[final_ultimate_data_df.GW == i].copy()
  working_df = working_df[working_df['total_points']>0]
  working_df['team'] = working_df['team'].astype(dtype="category")
  working_df['team'] =  working_df.team.cat.codes
  working_df['position'] = working_df['position'].apply(convertPosToNum)
  expected_scores = working_df["predicted_score_rf"]  # total points from last season
  prices = working_df["value"]/10
  positions = working_df["position"]
  clubs = working_df["team"]
  real_scores = working_df['total_points']
  # so we can read the results
  names = working_df["name"]
  decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)
  cum_decisions = {}
  cum_decisions['df'] = working_df
  cum_decisions['decisions'] = decisions
  cum_decisions['names'] = names
  cum_decisions['captain'] =  captain_decisions
  cum_decisions['sub_decisions'] = sub_decisions
  cum_decisions['prices'] = prices
  cum_decisions['position'] = positions
  cum_decisions['clubs'] = clubs
  cum_decisions['real_scores'] = real_scores
  cum_decisions['expected_scores'] = expected_scores
  final_decisions.append(cum_decisions)

In [ ]:
def team_selected(gw,df_mean,decisions,sub_decisions,captain_decisions, names,expected_scores, prices,real_scores,positions):
  print(f'{Fore.LIGHTBLUE_EX}{Style.BRIGHT}Our Final playing XI as predicted by the model:')
  print(gw)
  print("")

  names_list = []
  expected_list = []
  curr_budget = 0
  player_indices = []
  total_points = {}
  tpoints= 0
  rpoints =0
  df = pd.DataFrame()
  
  for i in range(df_mean.shape[0]):
      
      if decisions[i].value() != 0:
          xxy ={}
          xxy['name'] = names.iloc[i]
          xxy['team'] = merged_gw_21_22[(merged_gw_21_22.name == names.iloc[i])]['team'].values[0]
          xxy['expected_score'] = expected_scores.iloc[i]
          xxy['real_score'] = real_scores.iloc[i]
          xxy['position'] = convertNumToPos(positions.iloc[i])
          xxy['cost'] = prices.iloc[i]
          xxy['sub'] = 0
          xxy['non_sub'] = 1
          xxy['captain'] = 0
          xxy['GW'] = gw
          names_list.append(names.iloc[i])
          expected_list.append(expected_scores.iloc[i])
          total_points[names.iloc[i]] = expected_scores.iloc[i]
          player_indices.append(i)
          curr_budget+= prices.iloc[i]
          tpoints += expected_scores.iloc[i]
          rpoints += real_scores.iloc[i]
          display(Markdown("**Name**= *{}*, **Expected Points** = {},**Real Points** = {}, **Cost** = {}M$, **Position** = {}".format(names.iloc[i], expected_scores.iloc[i],real_scores.iloc[i], prices.iloc[i],  convertNumToPos(positions.iloc[i]))))
          if captain_decisions[i].value() == 1:
            xxy['captain'] = 1
            tpoints += expected_scores.iloc[i]
            rpoints += real_scores.iloc[i]
          df = df.append(xxy,ignore_index=True)


  print("")
  print(f'{Fore.LIGHTBLUE_EX}{Style.BRIGHT}Captain:')
  for i in range(df_mean.shape[0]):
      if captain_decisions[i].value() == 1:
          display(Markdown("**Name**= *{}*, **Expected Points** = {},**Real Points** = {}, **Cost** = {}M$, **Position** = {}".format(names.iloc[i], expected_scores.iloc[i],real_scores.iloc[i], prices.iloc[i],  convertNumToPos(positions.iloc[i]))))


  print("")
  print(f'{Fore.LIGHTBLUE_EX}{Style.BRIGHT}List of Substitutes:')

  for i in range(df_mean.shape[0]):
      if sub_decisions[i].value() == 1:
          names_list.append(names.iloc[i])
          player_indices.append(i)
          curr_budget+= prices.iloc[i]
          xxy ={}
          xxy['name'] = names.iloc[i]
          xxy['team'] = merged_gw_21_22[(merged_gw_21_22.name == names.iloc[i])]['team'].values[0]
          xxy['expected_score'] = expected_scores.iloc[i]
          xxy['real_score'] = real_scores.iloc[i]
          xxy['position'] = convertNumToPos(positions.iloc[i])
          xxy['cost'] = prices.iloc[i]
          xxy['sub'] = 1
          xxy['non_sub'] = 0
          xxy['captain'] = 0
          xxy['GW'] = gw
          df = df.append(xxy,ignore_index=True)
          # tpoints += expected_scores.iloc[i]
          display(Markdown("**Name**= *{}*, **Expected Points** = {},**Real Points** = {}, **Cost** = {}M$, **Position** = {}".format(names.iloc[i], expected_scores.iloc[i],real_scores.iloc[i], prices.iloc[i],  convertNumToPos(positions.iloc[i]))))
  
  print("")
  print(f'{Fore.LIGHTBLUE_EX}{Style.BRIGHT}Scores:')
  display(Markdown("**Predicted**= *{}*".format(tpoints)))
  display(Markdown("**Real**= *{}*".format(rpoints)))
  print("")
  return df
  

In [ ]:
count =1;
final_decisions_df = pd.DataFrame()
for i in final_decisions:
  x = team_selected(count,i['df'], i['decisions'], i['sub_decisions'],i['captain'],i['names'], i['expected_scores'],i['prices'],i['real_scores'],i['position'])
  final_decisions_df = final_decisions_df.append(x)
  count+=1


In [ ]:
final_decisions_df
import pickle
final_decisions_df.to_pickle('final_decisions_df.pkl')
final_decisions_df2 = pd.read_pickle('final_decisions_df.pkl')
final_decisions_df2.isna().sum()

In [ ]:
fig, ax = plt.subplots()
index = np.arange(12)
bar_width = 0.35
opacity = 0.8

r_sum = []
for i in range(1,13):
  gw_data = final_decisions_df[final_decisions_df.GW == i]
  r_sum.append(sum(gw_data['real_score']))

t_sum = []
for i in range(1,13):
  gw_data = final_decisions_df[final_decisions_df.GW == i]
  t_sum.append(sum(gw_data['expected_score']))

rects1 = plt.bar(index, r_sum, bar_width,
alpha=opacity,
color='b',
label='Actual Score')

rects2 = plt.bar(index + bar_width, t_sum, bar_width,
alpha=opacity,
color='g',
label='Predicted Score')

plt.xlabel('Predicted vs Actual Scores of players over Gameweeks')
plt.ylabel('Scores')
plt.title('Scores by players')
plt.xticks(index + bar_width, (1,2,3,4,5,6,7,8,9,10,11,12))
plt.legend()

plt.tight_layout()
plt.show()